In [1]:
from pyLCIO import IOIMPL, EVENT, UTIL
from os import environ as env
import os.path as osp

Loading LCIO ROOT dictionaries ...


In [2]:
reader = IOIMPL.LCFactory.getInstance().createLCReader()
reader.open(osp.abspath(f'{env["REPO_ROOT"]}/scripts/CPID.slcio'))

In [3]:
#col_pfo2mc = event.getCollection('RecoMCTruthLink')
#rel_pfo2mc = UTIL.LCRelationNavigator(col_pfo2mc)

In [4]:
event = reader.readNextEvent()

col = event.getCollection('PandoraPFOs')
params = col.getParameters()

PIDAlgNames = EVENT.StringVec()
PIDAlgIDs   = EVENT.IntVec()

params.getStringVals('PIDAlgorithmTypeName', PIDAlgNames)
params.getIntVals('PIDAlgorithmTypeID', PIDAlgIDs)

cpid_name = 'TMVA_BDT_MC_12bins_singleP'
cpid_idx = -1

for i, alg_name in enumerate(PIDAlgNames):
    if alg_name == cpid_name:
        cpid_idx = PIDAlgIDs[i]
        break

def extract_el_mu_ness(pfo, cpid_idx):
    pandora_type = pfo.getType()
    cpid_11ness = 0.
    cpid_13ness = 0.
    cpid_type = 0
    
    for pid in pfo.getParticleIDs():
        if pid.getAlgorithmType() == cpid_idx:
            cpid_params = pid.getParameters()
            cpid_11ness = float(cpid_params[0])
            cpid_13ness = float(cpid_params[1])
            cpid_type = pid.getType()
            break
        
    return pandora_type, cpid_type, cpid_11ness, cpid_13ness

In [ ]:
# from https://github.com/iLCSoft/MarlinReco/blob/e6c420abe8c80814fbbd3d34641b14dc90a23f5f/Analysis/PIDTools/src/ComprehensivePIDProcessor.cc

rel_mc2pfo = UTIL.LCRelationNavigator(event.getCollection('MCTruthRecoLink'))

truth_pdg = []
pandora_pdg = []
cpid_pdg = []
cpid_11_ness = []
cpid_13_ness = []

n_events = 100

bla = None

for n_evt in range(n_events):
    event = reader.readNextEvent()
    
    if event is None:
        break
    
    for mcp in event.getCollection('MCParticle'):
        if abs(mcp.getPDG()) in [11, 13]:
            mcp_pdg = abs(mcp.getPDG())
            bestwei = 0
            best_idx = -1
            
            pfovec = rel_mc2pfo.getRelatedToObjects(mcp)
            weivec = rel_mc2pfo.getRelatedToWeights(mcp)
            
            bla = pfovec
            
            for i in range(len(weivec)):
                trkwei = (int(weivec[i])%10000)/1000.
                cluwei = (int(weivec[i])/10000)/1000.
                
                mcparwei = trkwei if trkwei>cluwei else cluwei
                weight = weivec[i]
                
                if weight > bestwei:
                    best_idx = i
                    
            if best_idx != -1 and :
                truth_pdg.append(mcp_pdg)
                
                pandora_type, cpid_type, cpid_11ness, cpid_13ness = extract_el_mu_ness(pfovec[best_idx].getTo(), cpid_idx)
                pandora_pdg.append(pandora_type)
                cpid_pdg.append(cpid_type)
                cpid_11_ness.append(cpid_11ness)
                cpid_11_ness.append(cpid_13ness)

AttributeError: 'ParticleIDIOImpl' object has no attribute 'getTo'

In [17]:
t = pfovec[best_idx]

In [16]:
mcp.getEnergy()

0.9791661466039452

In [ ]:
IOIMPL.MCParticleIOImpl(bla[0])

TypeError: none of the 3 overloaded methods succeeded. Full details:
  MCParticleIOImpl::MCParticleIOImpl(IOIMPL::MCParticleIOImpl&&) =>
    ValueError: could not convert argument 1 (object is not an rvalue)
  MCParticleIOImpl::MCParticleIOImpl(const IOIMPL::MCParticleIOImpl&) =>
    TypeError: could not convert argument 1
  MCParticleIOImpl::MCParticleIOImpl() =>
    TypeError: takes at most 0 arguments (1 given)

In [ ]:
rel_mc2pfo = UTIL.LCRelationNavigator(event.getCollection('MCTruthRecoLink'))

truth_pdg = []
pandora_pdg = []
cpid_pdg = []
cpid_11_ness = []
cpid_13_ness = []

n_events = 100

for n_evt in range(n_events):
    event = reader.readNextEvent()
    
    if event is None:
        break
    
    for mcp in event.getCollection('MCParticle'):
        if abs(mcp.getPDG()) in [11, 13]:
            mcp_pdg = abs(mcp.getPDG())
            bestwei = 0
            best_idx = -1
            
            pfovec = rel_mc2pfo.getRelatedToObjects(mcp)

: 

In [ ]:
1 + 1

In [29]:
params.getStringVals('ParameterNames_TMVA_BDT_MC_12bins_singleP', string_vec)

vector<string>{ "11-ness", "13-ness", "211-ness", "321-ness", "2212-ness" }

In [ ]:
"11-ness", "13-ness", "211-ness", "321-ness", "2212-ness"

In [11]:
reader.close()

In [ ]:
r

result = []

for event in (pbar := tqdm(range(reader.getNumberOfEvents()))):
    event = reader.readNextEvent()
    pbar.set_description(f'Processing event {event.getEventNumber()}')

    # Iterate through the MCParticle collection and find muon and anti muon
    # see page 5 on https://pdg.lbl.gov/2019/reviews/rpp2019-rev-monte-carlo-numbering.pdf
    iso_leptons = event.getCollection('ISOLeptons')
    
    E = 0
    pX = 0
    pY = 0
    pZ = 0
    n_muons = 0
    
    for iso_lepton in iso_leptons:
        if abs(iso_lepton.getType()) == 13:
            mom = iso_lepton.getMomentum()
            
            E += iso_lepton.getEnergy()
            pX += mom[0]
            pY += mom[1]
            pZ += mom[2]
            n_muons += 1

    if n_muons == 2:
        result.append([E, pX, pY, pZ])
        
    # break

reader.getNumberOfEvents()